In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 8)
!date

%load_ext autoreload
%autoreload 2

Wed Feb 12 08:26:36 PST 2020


# Generate results and store them in json files

In [2]:
import dp_2020_census.data, dp_2020_census.model

In [3]:
df_orig = dp_2020_census.data.load_orig_counts()

In [4]:
epsilon = '1.0'
run = 1
df_new = dp_2020_census.data.load_dp_counts(epsilon, run)

In [5]:
stratification_cols = ['state', 'county', 'enum_dist',]
error = dp_2020_census.model.perturbation_error(df_orig, df_new, stratification_cols)

In [6]:
error.index = df_orig.groupby(stratification_cols)['count'].sum()

In [7]:
tc_error_enum_dist = np.median(np.absolute(error))
tc_error_enum_dist

15.0

In [8]:
np.percentile(np.absolute(error), 95)

52.0

In [9]:
def find_dp_mae(stratification_cols=['state', 'county', 'enum_dist'], result_name='tc_enum_dist'):
    results = {}

    df_orig = dp_2020_census.data.load_orig_counts()


    for epsilon in '0.25 0.50 0.75 1.0 2.0 4.0 6.0 8.0'.split():
        epsilon_underscore = epsilon.replace('.', '_')
        key = f'{result_name}_{epsilon_underscore}'
        print(key, end=' - ')
    
        error = []

        for run in '1234':
            df_new = dp_2020_census.data.load_dp_counts(epsilon, run)
            error_run = dp_2020_census.model.perturbation_error(df_orig, df_new, stratification_cols)
            error += list(error_run)
        
        mae = np.median(np.absolute(error))
        pct95 = np.quantile(np.absolute(error), 0.95)
        max_ae = max(np.absolute(error))
        rmse = np.sqrt(np.mean(np.array(error)**2))
        
        results[f'{key}_mae'] = f'{mae:.0f}'
        results[f'{key}_pct95'] = f'{pct95:.0f}'
        results[f'{key}_max_ae'] = f'{max_ae:.0f}'
        results[f'{key}_rmse'] = f'{rmse:.0f}'
        
        print(np.round(np.mean(error),2), np.round(mae,2),
              np.round(rmse, 2), np.round(pct95,1),
              max_ae)

    return results
find_dp_mae(['state', 'county'], 'tc_county')

tc_county_0_25 - 0.0 81.0 140.35 277.0 1020
tc_county_0_50 - 0.0 45.0 76.82 149.0 656
tc_county_0_75 - 0.0 31.0 52.76 103.0 395
tc_county_1_0 - 0.0 24.0 41.07 81.0 397
tc_county_2_0 - 0.0 13.0 22.02 43.0 210
tc_county_4_0 - 0.0 7.0 11.5 22.0 88
tc_county_6_0 - 0.0 5.0 7.93 16.0 66
tc_county_8_0 - 0.0 4.0 6.44 12.0 185


{'tc_county_0_25_mae': '81',
 'tc_county_0_25_pct95': '277',
 'tc_county_0_25_max_ae': '1020',
 'tc_county_0_25_rmse': '140',
 'tc_county_0_50_mae': '45',
 'tc_county_0_50_pct95': '149',
 'tc_county_0_50_max_ae': '656',
 'tc_county_0_50_rmse': '77',
 'tc_county_0_75_mae': '31',
 'tc_county_0_75_pct95': '103',
 'tc_county_0_75_max_ae': '395',
 'tc_county_0_75_rmse': '53',
 'tc_county_1_0_mae': '24',
 'tc_county_1_0_pct95': '81',
 'tc_county_1_0_max_ae': '397',
 'tc_county_1_0_rmse': '41',
 'tc_county_2_0_mae': '13',
 'tc_county_2_0_pct95': '43',
 'tc_county_2_0_max_ae': '210',
 'tc_county_2_0_rmse': '22',
 'tc_county_4_0_mae': '7',
 'tc_county_4_0_pct95': '22',
 'tc_county_4_0_max_ae': '88',
 'tc_county_4_0_rmse': '12',
 'tc_county_6_0_mae': '5',
 'tc_county_6_0_pct95': '16',
 'tc_county_6_0_max_ae': '66',
 'tc_county_6_0_rmse': '8',
 'tc_county_8_0_mae': '4',
 'tc_county_8_0_pct95': '12',
 'tc_county_8_0_max_ae': '185',
 'tc_county_8_0_rmse': '6'}

In [10]:
results = {}

results.update(find_dp_mae(['state'], 'tc_state'))
results.update(find_dp_mae(['state', 'county'], 'tc_county'))
results.update(find_dp_mae(['state', 'county', 'enum_dist'], 'tc_enum_dist'))

results.update(find_dp_mae(['state', 'age', 'race', 'eth'], 'sc_state'))
results.update(find_dp_mae(['state', 'county', 'age', 'race', 'eth'], 'sc_county'))
results.update(find_dp_mae(['state', 'county', 'enum_dist', 'age', 'race', 'eth'], 'sc_enum_dist'))

len(results)

tc_state_0_25 - 0.0 0.0 0.0 0.0 0
tc_state_0_50 - 0.0 0.0 0.0 0.0 0
tc_state_0_75 - 0.0 0.0 0.0 0.0 0
tc_state_1_0 - 0.0 0.0 0.0 0.0 0
tc_state_2_0 - 0.0 0.0 0.0 0.0 0
tc_state_4_0 - 0.0 0.0 0.0 0.0 0
tc_state_6_0 - 0.0 0.0 0.0 0.0 0
tc_state_8_0 - 0.0 0.0 0.0 0.0 0
tc_county_0_25 - 0.0 81.0 140.35 277.0 1020
tc_county_0_50 - 0.0 45.0 76.82 149.0 656
tc_county_0_75 - 0.0 31.0 52.76 103.0 395
tc_county_1_0 - 0.0 24.0 41.07 81.0 397
tc_county_2_0 - 0.0 13.0 22.02 43.0 210
tc_county_4_0 - 0.0 7.0 11.5 22.0 88
tc_county_6_0 - 0.0 5.0 7.93 16.0 66
tc_county_8_0 - 0.0 4.0 6.44 12.0 185
tc_enum_dist_0_25 - 0.0 56.0 93.11 190.0 688
tc_enum_dist_0_50 - 0.0 29.0 49.12 100.0 369
tc_enum_dist_0_75 - 0.0 20.0 33.66 68.0 285
tc_enum_dist_1_0 - 0.0 15.0 25.65 52.0 344
tc_enum_dist_2_0 - 0.0 8.0 13.49 27.0 115
tc_enum_dist_4_0 - 0.0 4.0 7.1 14.0 57
tc_enum_dist_6_0 - 0.0 3.0 4.86 10.0 45
tc_enum_dist_8_0 - 0.0 2.0 3.72 7.0 39
sc_state_0_25 - 0.0 18.0 49.53 110.0 348.0
sc_state_0_50 - 0.0 13.0 28.92 61

192

In [11]:
import json
with open('/ihme/homes/abie/projects/2019/dp_2020_census/results_dp_error.json', 'w') as f:
    json.dump(results, f, indent=4)

In [12]:
results['tc_enum_dist_0_25_mae']

'56'

# Confirm that the total count invariant worked at the state level

In [13]:
state_tc = find_dp_mae(['state',], 'tc_state')
pd.Series(state_tc)

tc_state_0_25 - 0.0 0.0 0.0 0.0 0
tc_state_0_50 - 0.0 0.0 0.0 0.0 0
tc_state_0_75 - 0.0 0.0 0.0 0.0 0
tc_state_1_0 - 0.0 0.0 0.0 0.0 0
tc_state_2_0 - 0.0 0.0 0.0 0.0 0
tc_state_4_0 - 0.0 0.0 0.0 0.0 0
tc_state_6_0 - 0.0 0.0 0.0 0.0 0
tc_state_8_0 - 0.0 0.0 0.0 0.0 0


tc_state_0_25_mae       0
tc_state_0_25_pct95     0
tc_state_0_25_max_ae    0
tc_state_0_25_rmse      0
tc_state_0_50_mae       0
                       ..
tc_state_6_0_rmse       0
tc_state_8_0_mae        0
tc_state_8_0_pct95      0
tc_state_8_0_max_ae     0
tc_state_8_0_rmse       0
Length: 32, dtype: object

In [14]:
assert np.all(pd.Series(state_tc).astype(float) == 0), \
    'State invariant should make total counts at state-level be zero error'

# Now calculate the empirical privacy loss

In [15]:
def find_dp_epl(stratification_cols=['state', 'county', 'enum_dist'], result_name='tc_privacy_loss_enum_dist'):
    results = {}

    df_orig = dp_2020_census.data.load_orig_counts()


    for epsilon in '0.25 0.50 0.75 1.0 2.0 4.0 6.0 8.0'.split():
        epsilon_underscore = epsilon.replace('.', '_')
        key = f'{result_name}_{epsilon_underscore}'
        print(key, end=' - ')
    
        error = []

        for run in '1234':
            df_new = dp_2020_census.data.load_dp_counts(epsilon, run)
            error_run = dp_2020_census.model.perturbation_error(df_orig, df_new, stratification_cols)
            error += list(error_run)
        
        error = pd.Series(error)
        df_epl = dp_2020_census.model.empirical_privacy_loss(error, bandwidth=.1,
                                                       est_range_percentile=95, est_range_multiplier=1.0)
        max_abs = max(np.absolute(df_epl.smooth_epl))

        results[f'{key}_max_abs'] = f'{max_abs:.3f}'
        
        print(results[f'{key}_max_abs'])

    return results
# find_dp_epl(['state',], 'tc_privacy_loss_state')  # this fails because of invariant

In [16]:
results = {}

# results.update(find_dp_epl(['state'], 'tc_state'))
results.update(find_dp_epl(['state', 'county'], 'tc_privacy_loss_county'))
results.update(find_dp_epl(['state', 'county', 'enum_dist'], 'tc_privacy_loss_enum_dist'))

results.update(find_dp_epl(['state', 'age', 'race', 'eth'], 'sc_privacy_loss_state'))
results.update(find_dp_epl(['state', 'county', 'age', 'race', 'eth'], 'sc_privacy_loss_county'))
results.update(find_dp_epl(['state', 'county', 'enum_dist', 'age', 'race', 'eth'],
                           'sc_privacy_loss_enum_dist'))

with open('/ihme/homes/abie/projects/2019/dp_2020_census/results_dp_epl.json', 'w') as f:
    json.dump(results, f, indent=4)

tc_privacy_loss_county_0_25 - 

/ihme/homes/abie/projects/2019/dp_2020_census/dp_2020_census/model.py:85: RuntimeWarning: divide by zero encountered in log
  df['epl_cusum'] = np.log(ccusum[1:] / ccusum[:-1])


0.014
tc_privacy_loss_county_0_50 - 0.035
tc_privacy_loss_county_0_75 - 0.038
tc_privacy_loss_county_1_0 - 0.048
tc_privacy_loss_county_2_0 - 0.094
tc_privacy_loss_county_4_0 - 0.169
tc_privacy_loss_county_6_0 - 0.225
tc_privacy_loss_county_8_0 - 0.318
tc_privacy_loss_enum_dist_0_25 - 0.017
tc_privacy_loss_enum_dist_0_50 - 0.033
tc_privacy_loss_enum_dist_0_75 - 0.046
tc_privacy_loss_enum_dist_1_0 - 0.064
tc_privacy_loss_enum_dist_2_0 - 0.116
tc_privacy_loss_enum_dist_4_0 - 0.219
tc_privacy_loss_enum_dist_6_0 - 0.299
tc_privacy_loss_enum_dist_8_0 - 0.381
sc_privacy_loss_state_0_25 - 0.119
sc_privacy_loss_state_0_50 - 0.104
sc_privacy_loss_state_0_75 - 0.166
sc_privacy_loss_state_1_0 - 0.166
sc_privacy_loss_state_2_0 - 0.300
sc_privacy_loss_state_4_0 - 0.453
sc_privacy_loss_state_6_0 - 0.541
sc_privacy_loss_state_8_0 - 0.742
sc_privacy_loss_county_0_25 - 0.126
sc_privacy_loss_county_0_50 - 0.165
sc_privacy_loss_county_0_75 - 0.150
sc_privacy_loss_county_1_0 - 0.164
sc_privacy_loss_county